In [1]:
import os
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import rasterio  # For geospatial data
from datetime import datetime
import faiss

# Define directories
image_dir = "data/geospatial"
csv_dir = "data/csv"
text_dir = "data/text"


/Users/azou/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/azou/.pyenv/versions/3.11.7/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [14]:
import rasterio
import numpy as np

image_data = {}  # Dictionary to store images by location and date
for filename in os.listdir(image_dir):
    with rasterio.open(os.path.join(image_dir, filename)) as src:
        raster_data = src.read(1)  # Read the first band
        metadata = src.meta

        print("Metadata:", metadata)
        print("Data shape:", raster_data.shape)

        # Example: Show coordinate reference system
        crs = src.crs
        print("CRS:", crs)

        name = filename.split(".")[0]
        image_data[(crs, name)] = raster_data

print("Successfully processed tif data of length: ", len(image_data))

Metadata: {'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 4096, 'height': 4096, 'count': 12, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(0.0025, 0.0, 40.965,
       0.0, -0.0025, 11.99)}
Data shape: (4096, 4096)
CRS: EPSG:4326
Metadata: {'driver': 'GTiff', 'dtype': 'int16', 'nodata': None, 'width': 4096, 'height': 5475, 'count': 12, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(

In [22]:
from torchvision import models, transforms

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ResNet model
resnet_model = models.resnet50(pretrained=True)
resnet_model = torch.nn.Sequential(
    *(list(resnet_model.children())[:-1])
)  # Remove final classification layer
resnet_model = resnet_model.to(device)
resnet_model.eval()

# Define preprocessing
preprocess = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Image embedding function
def get_image_embedding(image_array):
    # Ensure raster data is in a suitable format for preprocessing
    if image_array.dtype != np.uint8:
        image_array = (
            255 * (image_array - image_array.min()) / (image_array.ptp())
        ).astype(np.uint8)

    # Stack the image to 3 channels to match ResNet's input
    image_array = np.stack([image_array] * 3, axis=-1)
    processed_image = (
        preprocess(image_array).unsqueeze(0).to(device)
    )  # Move to GPU if available

    # Generate embedding
    with torch.no_grad():
        embedding = resnet_model(processed_image).squeeze().cpu().numpy()
    return embedding


# Create embeddings for each image and store in a dictionary
image_embeddings = {}
for (crs, name), raster_data in image_data.items():
    embedding = get_image_embedding(raster_data)
    image_embeddings[(crs, name)] = embedding

print(f"Successfully added embeddings for {len(image_embeddings)} data")

/Users/azou/.pyenv/versions/3.11.7/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/azou/.pyenv/versions/3.11.7/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Successfully added embeddings for 4 data


In [ ]:
# Convert embeddings to a numpy array for FAISS and keep a list of metadata
embedding_matrix = np.array(list(image_embeddings.values())).astype("float32")
image_keys = list(image_embeddings.keys())  # Store the (crs, name) keys for reference

# Initialize FAISS index for L2 distance (Euclidean distance)
dimension = embedding_matrix.shape[1]  # Number of features in each embedding
index = faiss.IndexFlatL2(dimension)
index.add(embedding_matrix)  # Add all embeddings to the index

# Check that the index has been populated
print("Number of images indexed:", index.ntotal)

# Semantic search function to retrieve top-k similar images
def semantic_search(query_image_array, k=5):
    # Generate embedding for the query image
    query_embedding = get_image_embedding(query_image_array).astype("float32")

    # Perform FAISS search
    D, I = index.search(np.array([query_embedding]), k)  # D = distances, I = indices

    # Retrieve top-k metadata based on indices
    top_k_files = [image_keys[idx] for idx in I[0]]
    top_k_distances = D[0]

    return list(zip(top_k_files, top_k_distances))  # Return metadata with distances


# Example search for image similarity (using an existing image as query)
query_image_key = list(image_data.keys())[0]  # Taking the first image as an example
query_image = image_data[query_image_key]  # Get the raster data

results = semantic_search(query_image, k=5)
print("Top 5 similar images:", results)

Number of images indexed: 4
Top 5 similar images: [((CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'drought-vulnerability-summary_0'), 0.0), ((CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'drought-vulnerability-summary_2'), 157.74901), ((CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251

In [25]:
from fastapi import FastAPI, HTTPException

app = FastAPI()

# Endpoint for semantic image search (using an image file as input)
@app.post("/search_image/")
async def search_image(k: int = 5):
    try:
        # Process image (this part would require proper decoding if input is raw bytes)
        results = semantic_search(query_image, k=5)
        return {"top_matches": results}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run with `uvicorn <filename>:app --reload`